# Text generation

In [1]:
import github_command as gt

In [2]:
gt.push(file_to_transfer="TD7_Text_Generation_With_LSTM.ipynb",
       message="text generation",
       repos="TDs_ESILV.git")

## Load Packages

In [4]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [11]:
!pip install nltk

In [14]:
import nltk
from nltk.text import Text

In [22]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/lucbertin/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [28]:
alice = nltk.corpus.gutenberg.words('carroll-alice.txt')

In [53]:
# load ascii text and covert to lowercase
#filename = "wonderland.txt"
#raw_text = open(filename, 'r', encoding='utf-8').read()
#raw_text = raw_text.lower()
raw_text = " ".join(alice).lower()

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: {}".format(n_chars))
print("Total Vocab: {}".format(n_vocab))

# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    #print(seq_in)
    seq_out = raw_text[i + seq_length]
    #print(seq_out)
    dataX.append([char_to_int[char] for char in seq_in])
    #print(dataX)
    dataY.append(char_to_int[seq_out])
    #print(dataY)
    

n_patterns = len(dataX)
print("Total Patterns: {}".format(n_patterns))

### 
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Characters: 150118
Total Vocab: 46
Total Patterns: 150018


### Define the LSTM model

In [54]:
X.shape, y.shape

((150018, 100, 1), (150018, 46))

In [55]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 46)                11822     
Total params: 276,014
Trainable params: 276,014
Non-trainable params: 0
_________________________________________________________________


### Define the checkpoint

In [61]:
X.shape

(150018, 100, 1)

In [57]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [58]:
hist = model.fit(X, y, epochs=2, batch_size=128, callbacks=[checkpoint])

Epoch 1/2
150018/150018 [==============================] - 343s 2ms/step - loss: 2.7943

Epoch 00001: loss improved from inf to 2.79430, saving model to weights-improvement-01-2.7943.hdf5
Epoch 2/2
150018/150018 [==============================] - 339s 2ms/step - loss: 2.5352

Epoch 00002: loss improved from 2.79430 to 2.53518, saving model to weights-improvement-02-2.5352.hdf5


In [ ]:
for i in range(1,100):
    model.predict(x=([]))